In [20]:
# Implementation of Poseidon Hash function
import json

P = 21888242871839275222246405745257275088548364400416034343698204186575808495617
POSEIDON_CONST = json.load(open('poseidon_constants.json'))
N_ROUNDS_P = [56, 57, 56, 60, 60, 63, 64, 63, 60, 66, 60, 65, 70, 60, 64, 68]

def poseidon(*inputs):
    t = len(inputs) + 1
    state = [0] + list(inputs)
    for r in range(8 + N_ROUNDS_P[t-2]):
        state = [(s + POSEIDON_CONST['C'][t-2][r*t + i]) % P for i, s in enumerate(state)]
        if r < 4 or r >= 4 + N_ROUNDS_P[t-2]:
            state = [s**5 % P for s in state]
        else:
            state[0] = state[0]**5%P
        state = [sum(POSEIDON_CONST['M'][t-2][i][j] * s for j, s in enumerate(state)) % P for i in range(t)]
    return state[0] % P

assert poseidon(11, 0) == 1450217488996495680417999281110793894108725512014359364483255385094537306690
assert poseidon(
    7728219356619767300678996689460199020238374873464770777330570693751508016209,
    3142451648954394489869726249513242327747495537215163198503777895701803500487,
    16508917144752610602145963506823743115557101240265470506805505298395529637033,
    18631654747796370155722974221085383534170330422926471002342567715267253236113,
    420690123456789
) == 16077843655500990663898521995184914806093173419143170117384200584662255685399

In [21]:
# Implementation of Lean Binary Incremental Merkle Tree
# https://zkkit.pse.dev/classes/_zk_kit_lean_imt.LeanIMT.html

# hashFn is any two-argument hash function (such as poseidon)
def leanIMT(hashFn, items):
    while len(items) > 1:
        items = [hashFn(items[i], items[i+1]) if i+1 < len(items) else items[i]
                 for i in range(0, len(items), 2)]
    return items[0]

assert leanIMT(poseidon, [
    446370619522020906414390803988708135454030711365737047028726600321713888240, 
    7061949393491957813657776856458368574501817871421526214197139795307327923534
]) == 12770390212471611000655936249702043405744538606769666268598077668392487442602
assert leanIMT(poseidon, [
    446370619522020906414390803988708135454030711365737047028726600321713888240, 
    7061949393491957813657776856458368574501817871421526214197139795307327923534,
      273045008416047310365567535238707127657106682225732927746664352533972939910, 
    290289384240466119421503119796692120643433371363968266763205061964021262113
]) == 17005993619935710775202459447926005298591311681297711148569033759123477678563

In [22]:
# Python implementation of BLAKE-512 hash function
import struct

sigma = [
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    [14, 10, 4, 8, 9, 15, 13, 6, 1, 12, 0, 2, 11, 7, 5, 3],
    [11, 8, 12, 0, 5, 2, 15, 13, 10, 14, 3, 6, 7, 1, 9, 4],
    [7, 9, 3, 1, 13, 12, 11, 14, 2, 6, 5, 10, 4, 0, 15, 8],
    [9, 0, 5, 7, 2, 4, 10, 15, 14, 1, 11, 12, 6, 8, 3, 13],
    [2, 12, 6, 10, 0, 11, 8, 3, 4, 13, 7, 5, 15, 14, 1, 9],
    [12, 5, 1, 15, 14, 13, 4, 10, 0, 7, 6, 3, 9, 2, 8, 11],
    [13, 11, 7, 14, 12, 1, 3, 9, 5, 0, 15, 4, 8, 6, 2, 10],
    [6, 15, 14, 9, 11, 3, 0, 8, 12, 2, 13, 7, 1, 4, 10, 5],
    [10, 2, 8, 4, 7, 6, 1, 5, 15, 11, 9, 14, 3, 12, 13, 0],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    [14, 10, 4, 8, 9, 15, 13, 6, 1, 12, 0, 2, 11, 7, 5, 3],
    [11, 8, 12, 0, 5, 2, 15, 13, 10, 14, 3, 6, 7, 1, 9, 4],
    [7, 9, 3, 1, 13, 12, 11, 14, 2, 6, 5, 10, 4, 0, 15, 8],
    [9, 0, 5, 7, 2, 4, 10, 15, 14, 1, 11, 12, 6, 8, 3, 13],
    [2, 12, 6, 10, 0, 11, 8, 3, 4, 13, 7, 5, 15, 14, 1, 9]
]

u512 = [
    0x243f6a8885a308d3, 0x13198a2e03707344,
    0xa4093822299f31d0, 0x082efa98ec4e6c89,
    0x452821e638d01377, 0xbe5466cf34e90c6c,
    0xc0ac29b7c97c50dd, 0x3f84d5b5b5470917,
    0x9216d5d98979fb1b, 0xd1310ba698dfb5ac,
    0x2ffd72dbd01adfb7, 0xb8e1afed6a267e96,
    0xba7c9045f12c7f99, 0x24a19947b3916cf7,
    0x0801f2e2858efc16, 0x636920d871574e69
]

padding = bytearray(129)
padding[0] = 0x80

def rotr64(x, n):
    return ((x >> n) | (x << (64 - n))) & 0xffffffffffffffff

def G(i, m, v, a, b, c, d, e):
    v[a] = (v[a] + (m[sigma[i][e]] ^ u512[sigma[i][e + 1]]) + v[b]) & 0xffffffffffffffff
    v[d] = rotr64(v[d] ^ v[a], 32)
    v[c] = (v[c] + v[d]) & 0xffffffffffffffff
    v[b] = rotr64(v[b] ^ v[c], 25)
    v[a] = (v[a] + (m[sigma[i][e + 1]] ^ u512[sigma[i][e]]) + v[b]) & 0xffffffffffffffff
    v[d] = rotr64(v[d] ^ v[a], 16)
    v[c] = (v[c] + v[d]) & 0xffffffffffffffff
    v[b] = rotr64(v[b] ^ v[c], 11)


class Blake512:
    def __init__(self):
        self.h = [
            0x6a09e667f3bcc908, 0xbb67ae8584caa73b,
            0x3c6ef372fe94f82b, 0xa54ff53a5f1d36f1,
            0x510e527fade682d1, 0x9b05688c2b3e6c1f,
            0x1f83d9abfb41bd6b, 0x5be0cd19137e2179
        ]
        self.s = [0] * 4
        self.t = [0] * 2
        self.buflen = 0
        self.nullt = False
        self.buf = bytearray(128)

    def compress(self, block):
        v = self.h[:] + [
            self.s[0] ^ u512[0], self.s[1] ^ u512[1],
            self.s[2] ^ u512[2], self.s[3] ^ u512[3],
            u512[4], u512[5], u512[6], u512[7]
        ]
        m = [struct.unpack('>Q', block[i:i+8])[0] for i in range(0, 128, 8)]
        if not self.nullt:
            v[12] ^= self.t[0]
            v[13] ^= self.t[0]
            v[14] ^= self.t[1]
            v[15] ^= self.t[1]
        for i in range(16):
            G(i, m, v, 0, 4, 8, 12, 0)
            G(i, m, v, 1, 5, 9, 13, 2)
            G(i, m, v, 2, 6, 10, 14, 4)
            G(i, m, v, 3, 7, 11, 15, 6)
            G(i, m, v, 0, 5, 10, 15, 8)
            G(i, m, v, 1, 6, 11, 12, 10)
            G(i, m, v, 2, 7, 8, 13, 12)
            G(i, m, v, 3, 4, 9, 14, 14)
        for i in range(16): self.h[i % 8] ^= v[i]
        for i in range(8): self.h[i] ^= self.s[i % 4]

    def update(self, data):
        left = self.buflen
        fill = 128 - left
        if left and len(data) >= fill:
            self.buf[left:] = data[:fill]
            self.t[0] += 1024
            if self.t[0] == 0:
                self.t[1] += 1
            self.compress(self.buf)
            data = data[fill:]
            left = 0
        while len(data) >= 128:
            self.t[0] += 1024
            if self.t[0] == 0: self.t[1] += 1
            self.compress(data[:128])
            data = data[128:]
        if data:
            self.buf[left:left+len(data)] = data
            self.buflen = left + len(data)
        else: self.buflen = 0

    def digest(self):
        out = bytearray(64)
        lo = self.t[0] + self.buflen * 8
        hi = self.t[1]
        if lo < self.buflen * 8: hi += 1
        msglen = struct.pack('>QQ', hi, lo)
        if self.buflen == 111:
            self.t[0] -= 8
            self.update(b'\x81')
        else:
            if self.buflen < 111:
                if not self.buflen: self.nullt = True
                self.t[0] -= 888 - self.buflen * 8
                self.update(padding[:111 - self.buflen])
            else:
                self.t[0] -= 1024 - self.buflen * 8
                self.update(padding[:128 - self.buflen])
                self.t[0] -= 888
                self.update(padding[1:112])
                self.nullt = True
            self.update(b'\x01')
            self.t[0] -= 8
        self.t[0] -= 128
        self.update(msglen)
        for i in range(8): struct.pack_into('>Q', out, i * 8, self.h[i])
        return out

def blake512(message):
    S = Blake512()
    S.update(message)
    return S.digest()

assert blake512(bytes([])).hex() == 'a8cfbbd73726062df0c6864dda65defe58ef0cc52a5625090fa17601e1eecd1b628e94f396ae402a00acc9eab77b4d4c2e852aaaa25a636d80af3fc7913ef5b8'

In [23]:
# Implementation of Baby Jubjub elliptic curve methods

# Extended euclidean algorithm
def inv(a, n):
    if a == 0: return 0
    lm, hm = 1, 0
    low, high = a % n, n
    while low > 1:
        r = high//low
        nm, new = hm-lm*r, high-low*r
        lm, low, hm, high = nm, new, lm, low
    return lm % n

Base8 = (
    5299619240641551281634865583518297030282874472190772894086521144482721001553,
    16950150798460657717958625567821834550301663161624707787222815936182638968203
)
Order = 21888242871839275222246405745257275088614511777268538073601725287587578984328
SubOrder = Order >> 3
BjA = 168700
BjD = 168696


def add_bj(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    x3 = ((x1*y2 + y1*x2) % P) * inv((1 + BjD*x1*x2*y1*y2) % P, P)
    y3 = ((y1*y2 - BjA*x1*x2) % P) * inv((P + 1 - BjD*x1*x2*y1*y2) % P, P)
    return (x3 % P, y3 % P)

def multiply_bj(pt, n):
    if n == 0: return None
    elif n == 1: return pt
    elif not n % 2: return multiply_bj(add_bj(pt, pt), n // 2)
    else: return add_bj(multiply_bj(add_bj(pt, pt), int(n // 2)), pt)
    

In [24]:
# Implementation of EdDSA

pm1d2 = (P - 1) * inv(2, P) % P

def pack_point(A):
    buff = bytearray(A[1].to_bytes(32, byteorder='little'))
    if A[0] > pm1d2: buff[31] = buff[31] | 0x80
    return buff

def eddsa_poseidon_sign(privateKey, message):
    sBuff = bytearray(blake512(privateKey))
    # pruneBuffer
    sBuff[0] = sBuff[0] & 0xF8
    sBuff[31] = sBuff[31] & 0x7F
    sBuff[31] = sBuff[31] | 0x40

    s = int.from_bytes(sBuff[:32], byteorder='little')
    A = multiply_bj(Base8, s >> 3)
    
    rBuff = blake512(sBuff[32:] + message.to_bytes(32, byteorder='little'))
    r = int.from_bytes(rBuff, byteorder='little') % SubOrder
    R8 = multiply_bj(Base8, r)
    hms = poseidon(R8[0], R8[1], A[0], A[1], message)
    S = (r + hms*s) % SubOrder
    
    return {
        'publicKey': pack_point(A),
        'signature': pack_point(R8) + S.to_bytes(32, byteorder='little')
    }

In [25]:
# Implementation of POD (Programmable Object Data) PCD (Proof Carrying Data)

import base64
import uuid
import hashlib

def pod_hash(value):
    if value['type'] == 'string':
        return int.from_bytes(hashlib.sha256(value['value'].encode('utf-8')).digest(), byteorder='big') >> 8
    elif value['type'] in ('int', 'cryptographic'):
        return poseidon(value['value'])

def create_pod_pcd(privateKey, data):
    hashes = []
    for k, v in data.items():
        hashes.append(pod_hash({'value': k, 'type': 'string'}))
        hashes.append(pod_hash(v))

    message = leanIMT(poseidon, hashes)
    sign = eddsa_poseidon_sign(privateKey, message)
    return json.dumps({
        'id': str(uuid.uuid4()),
        'claim': {
            'entries': data,
            'signerPublicKey': base64.b64encode(sign['publicKey']).decode('utf-8'),
        },
        'proof': {
             'signature': base64.b64encode(sign['signature']).decode('utf-8'),   
        }
    }, separators=(',', ':'))



In [26]:
# Generate URL for adding the PCD to Zupass
from urllib.parse import quote

def make_zupass_url(pcd, returnUrl="https://zupass.org/"):
    return 'https://zupass.org/#/add?request='+quote(json.dumps({
        "type": "Add",
        "returnUrl": returnUrl,
        "pcd": {
            "type": "pod-pcd",
            "pcd": pcd
        }
    },separators=(',', ':')))

In [27]:
# Generate a Zupass link for a POD PCD

private_key = b"\0" * 32
zupass_url = make_zupass_url(create_pod_pcd(private_key, {
    "attack":{"type":"int","value":7},
    "itemSet":{"type":"string","value":"celestial"},
    "pod_type":{"type":"string","value":"item.weapon"},
    "weaponType":{"type":"string","value":"sword"}
}))

print(zupass_url)

https://zupass.org/#/add?request=%7B%22type%22%3A%22Add%22%2C%22returnUrl%22%3A%22https%3A//zupass.org/%22%2C%22pcd%22%3A%7B%22type%22%3A%22pod-pcd%22%2C%22pcd%22%3A%22%7B%5C%22id%5C%22%3A%5C%2294adb815-9af3-41ab-a8d6-74864ddd0b13%5C%22%2C%5C%22claim%5C%22%3A%7B%5C%22entries%5C%22%3A%7B%5C%22attack%5C%22%3A%7B%5C%22type%5C%22%3A%5C%22int%5C%22%2C%5C%22value%5C%22%3A7%7D%2C%5C%22itemSet%5C%22%3A%7B%5C%22type%5C%22%3A%5C%22string%5C%22%2C%5C%22value%5C%22%3A%5C%22celestial%5C%22%7D%2C%5C%22pod_type%5C%22%3A%7B%5C%22type%5C%22%3A%5C%22string%5C%22%2C%5C%22value%5C%22%3A%5C%22item.weapon%5C%22%7D%2C%5C%22weaponType%5C%22%3A%7B%5C%22type%5C%22%3A%5C%22string%5C%22%2C%5C%22value%5C%22%3A%5C%22sword%5C%22%7D%7D%2C%5C%22signerPublicKey%5C%22%3A%5C%22kfEJWsAZtQYQtctW5ds4iRd/7otkIvyj2sBO4ZMkMak%3D%5C%22%7D%2C%5C%22proof%5C%22%3A%7B%5C%22signature%5C%22%3A%5C%22ko2gpTQMPhMAhKbm5Di66vDFem6TuLsMBpRlmvYz8pgAD%2BmO%2BPJkHiDghvUH2akISsxOhMi3L9npMuuMGE79BA%3D%3D%5C%22%7D%7D%22%7D%7D
